In [6]:
import numpy as np
import ArtistExtractor as AE
import pandas as pd
import glob
import os
import LocationUtils as LU
from ExtractGenreClubs import SplitLineup
from ast import literal_eval

In [7]:
import ArtistExtractor as AE
def generateDictionnary(str_dataframe):
    begin = 0
    end = 45378
    path = str_dataframe
    path = "FullData/ArtistDataframe.csv"
    df = pd.read_csv(path)
    df

    i=0
    AllGenresDownloaded = []
    print("Populating dictionnary with other genres:")
    print("Reading all genres from dataframe : "+path)
    for id,row in df.iterrows():
        listGenres = literal_eval(row[2])
        if(listGenres!=None):
            AllGenresDownloaded+=listGenres
    print("All genre extracted from dataframe. Updating dictionnary..")
    dic = AE.createDictionnary()
    dic = AE.updateDictionnary(AllGenresDownloaded,dic,debug=True)
    LU.saveDictionary(dic,"FullData/AllGenresDic",enc="utf-8")
    print("Dictionnary of genre saved.")

In [8]:
dictionnaryOfGenres = LU.loadDictionary("FullData/AllGenresDic",enc="utf-8")

In [9]:
def downloadGenresWikipedia(str_file,dictionnary,begin=0,end=100000):

    Artists = pd.read_csv(str_file,index_col=0)
    
    if(end>Artists.shape[0]):
        end = Artists.shape[0]
    
    path = "FullData/ArtistDataframe_"+str(begin)+"_"+str(end)+".csv"
    print("Downloading genres of "+str(end-begin)+" artist...")
    last_read = 0
    for i in range(begin,end):
        S = Artists[Artists.index==i]
        artist = S["artist"].values[0]
        genres_ra = AE.getGenresFromWikipedia(artist,dictionnary)
        if(len(genres_ra)<1):
            genres_ra = None
        Artists.loc[Artists.index==i,"genres_ra"] = str(genres_ra)

        if(i%300==0):
            Artists.to_csv(path,encoding="utf-8")
            print(str(i)+": Saved under "+path)
    
    Artists.to_csv(path,encoding="utf-8")
    
    print(str(i)+": Saved under "+path)
    last_read = i

    print("Last read = "+str(last_read))
    return Artists

In [ ]:
a = downloadGenresWikipedia("FullData/ArtistDataframe.csv",dictionnaryOfGenres,21000)
a

21000: Saved under FullData/ArtistDataframe_21000_45378.csv
21300: Saved under FullData/ArtistDataframe_21000_45378.csv
21600: Saved under FullData/ArtistDataframe_21000_45378.csv
21900: Saved under FullData/ArtistDataframe_21000_45378.csv
22200: Saved under FullData/ArtistDataframe_21000_45378.csv
22500: Saved under FullData/ArtistDataframe_21000_45378.csv
22800: Saved under FullData/ArtistDataframe_21000_45378.csv
23100: Saved under FullData/ArtistDataframe_21000_45378.csv
23400: Saved under FullData/ArtistDataframe_21000_45378.csv
23700: Saved under FullData/ArtistDataframe_21000_45378.csv
24000: Saved under FullData/ArtistDataframe_21000_45378.csv
24300: Saved under FullData/ArtistDataframe_21000_45378.csv
24600: Saved under FullData/ArtistDataframe_21000_45378.csv
24900: Saved under FullData/ArtistDataframe_21000_45378.csv
25200: Saved under FullData/ArtistDataframe_21000_45378.csv
25500: Saved under FullData/ArtistDataframe_21000_45378.csv
25800: Saved under FullData/ArtistDatafr

In [ ]:
file = "ClubData/ClubDataGeo/ClubDataFrame.csv"
ClubDataFrame = pd.read_csv(file,sep=",")
ClubDataFrame["Genre"] = None
ClubDataFrame["location"] = None
ClubDataFrame["club"] = 1
ClubDataFrame = ClubDataFrame[["ClubName","club","Address","location","EventName","Date","LineUp","Genre","Location"]]
ClubDataFrame.columns = ["place","club","address","location","event","date","artists","genre","coordinates"]
ClubDataFrame

In [ ]:
file = "EventsChData/completeWithCoordinates.csv"
EventsDataFrame = pd.read_csv(file)
EventsDataFrame["address"] = None
EventsDataFrame["place"] = None
EventsDataFrame["club"] = 0
EventsDataFrame = EventsDataFrame[["place","club","address","location","event","date","artists","genre","coordinates"]]
EventsDataFrame

In [ ]:
Dataframe = pd.DataFrame()
Dataframe = Dataframe.append(EventsDataFrame)
Dataframe = Dataframe.append(ClubDataFrame)
Dataframe.to_csv("FullData/EventsAndClubs.csv")

# Getting all artists

In [ ]:
ArtistsDataFrame = pd.DataFrame()
i=0
print(Dataframe.shape)
for index,row in Dataframe.iterrows():
    lineup = row[6] #get line-up
    artists = literal_eval(lineup)
    for a in artists:
        a = a.split("(")[0]#Remove parenthesis at end of artist (record label, live act..)
        a = pd.Series(a.strip())
        ArtistsDataFrame = ArtistsDataFrame.append(a,ignore_index=True)
    i+=1
    if(i%5000==0):
        print(i)
        
ArtistsDataFrame.columns=["artist"]
ArtistsDataFrame.to_csv("FullData/ArtistsList.csv")
ArtistsDataFrame.head(10)

In [ ]:
ArtistsDataFrame.to_csv("FullData/ArtistsList.csv",encoding="utf-8")

In [ ]:
def downloadGenres(str_file,begin=0,end=100000):

    Artists = pd.read_csv(str_file,index_col=0)
    Artists = Artists.drop_duplicates().reset_index()[["artist"]]
    Artists.columns=["artist"]
    Artists["genres_spotify"]=None
    Artists["genres_ra"]=None
    Artists["main_genres"]=None
    Artists["top3_genres"]=None
    Artists["genre"]=None    
    
    if(end>Artists.shape[0]):
        end = Artists.shape[0]
    
    path = "FullData/ArtistDataframe_"+str(begin)+"_"+str(end)+".csv"
    print("Downloading genres of "+str(end-begin)+" artist...")
    last_read = 0
    for i in range(begin,end):
        S = Artists[Artists.index==i]
        artist = S["artist"].values[0]
        genres_spotify = AE.getGenresFromSpotify(artist)
        if(len(genres_spotify)<1):
            genres_spotify = None
        Artists.loc[Artists.index==i,"genres_spotify"] = str(genres_spotify)

        if(i%300==0):
            Artists.to_csv(path,encoding="utf-8")
            print(str(i)+": Saved under "+path)
    
    Artists.to_csv(path,encoding="utf-8")
    
    print(str(i)+": Saved under "+path)
    last_read = i

    print("Last read = "+str(last_read))
    return Artists

In [ ]:
A = downloadGenres("FullData/ArtistsList.csv",0)
A.head(10)

In [ ]:
begin = 0
end = 45378

df = pd.read_csv("FullData/ArtistDataframe.csv")
df

In [ ]:
import ArtistExtractor as AE

begin = 0
end = 45378

df = pd.read_csv("FullData/ArtistDataframe.csv")
df

i=0
AllGenresDownloaded = []
for id,row in df.iterrows():
    listGenres = literal_eval(row[2])
    print(listGenres)
    if(listGenres!=None):
        AllGenresDownloaded+=listGenres
    i+=1
    if(i>10):
        break
print(AllGenresDownloaded)

dic = AE.createDictionnary()
AE.updateDictionnary(AllGenresDownloaded,dic,debug=True)

# Construct full artist dictionnary

In [ ]:
files = []
files = glob.glob("Artists/*.txt")
print("Nb files : "+str(len(files)))
ArtistsMainDic = dict()
ArtistsAllDic = dict()

for file in files :
    filename = file.replace(".txt","")
    dictTemp = LU.loadDictionary(filename,path="")
    
    if("All" in file):
        ArtistsAllDic = {**ArtistsAllDic,**dictTemp}
        
    if("Main" in file):
        ArtistsMainDic = {**ArtistsMainDic,**dictTemp}
        
print("Artist dictionaries loaded. Merging...",end="")
LU.saveDictionary(ArtistsAllDic,"ArtistsAllGenres","Artists/",enc="UTF-8")
LU.saveDictionary(ArtistsMainDic,"ArtistsMainGenre","Artists/",enc="UTF-8")
print("done.")
print("Deleting temp files...")
for file in files :
    if("-" in file and "temp" in file):
        print("Deleting temp file "+file)
        os.remove(file)
print("DONE.")

## Creating dataframe of artists

In [ ]:
ArtistDicoMain = LU.loadDictionary("ArtistsMainGenreOld","Artists/",enc="UTF-8")
ArtistDicoAll =  LU.loadDictionary("ArtistsAllGenresOld","Artists/",enc="UTF-8")
ArtistsSet = set(a.encode("UTF-8") for a in ArtistDicoMain.keys())

ArtistsSet.update(ArtistDicoAll.keys())

DictionnaryGenre = LU.loadDictionary("MusicGenres","Artists/",enc="UTF-8")

ClubDataFrame["Genre"] = None
ClubDataFrame["All Genres"] = None
print("Retrieving genre over " +str(ClubDataFrame.shape)+" events:")

MissingGenres = set()

i=0
for id,row in ClubDataFrame.iterrows():
    genres = []
    lineup = row["LineUp"]
    artists = SplitLineup(lineup)
    
    for artist in artists:
        mainGenre = None
        allGenres = None
        
        if(artist in ArtistsSet):
            mainGenre = ArtistDicoMain.get(artist)
            if(mainGenre!="None" and mainGenre!=None):
                if(mainGenre not in DictionnaryGenre):
                    MissingGenres.add(mainGenre)
                else:
                    translatedGenre = DictionnaryGenre[mainGenre]
                if(translatedGenre!=None):
                    mainGenre = translatedGenre
                allGenres = ArtistDicoAll.get(artist)
        else:
            print("Not found in dictionnary :"+str(artist),end = "")
            #Updating dictionnaries
            ArtistsSet.add(artist)
            allGenres = AE.getGenre(artist,dictionnary=DictionnaryGenre,ReturnAllGenres=True)
            mainGenre = AE.getMaxGenre(allGenres,)
            print(", MainGenre is : "+str(mainGenre))
            ArtistDicoMain.update({artist : mainGenre})
            ArtistDicoAll.update({artist : allGenres})
        
        #Adding to LineUp genres
        genres.append(mainGenre) 
        
            
    if(len(genres)==0):
        print(artists)
    else:
        maxGenre =AE.getMaxGenre(genres)
        #updating dataframe
        ClubDataFrame = ClubDataFrame.set_value(id,"Genre",maxGenre)
        ClubDataFrame = ClubDataFrame.set_value(id,"All Genres",str(genres))
        ClubDataFrame = ClubDataFrame.set_value(id,"LineUp",str(artists))
    
    i+=1
    if(i%200==0):
        print(str(i))
    if(i%200==0):
        LU.saveDictionary(ArtistDicoMain,"ArtistDicoMain",path = "Artists/",enc="UTF-8")
        LU.saveDictionary(ArtistDicoAll,"ArtistDicoAll",path = "Artists/",enc="UTF-8")
        
print("FINISHED")
LU.saveDictionary(ArtistDicoMain,"ArtistDicoMain",path = "Artists/",enc="UTF-8")
LU.saveDictionary(ArtistDicoAll,"ArtistDicoAll",path = "Artists/",enc="UTF-8")

print("\n\n These genres were not found in dictionary : ")
print(MissingGenres)

In [ ]:
AE.createDictionnary()
ClubDataFrame[0:50]

In [ ]:
ClubDataFrame.to_csv("ClubData/EventsWithGenre.csv",encoding="UTF-8")